In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

def show_img(img,cmap='gray'):
    fig = plt.figure(figsize=(15, 15))
    ax = fig.add_subplot(111)
    ax.imshow(img, cmap=cmap)

In [ ]:
one_box = cv2.imread('../../data/reeses_puffs.png',0)
# plt.imshow(one_box,cmap='gray')

many_box = cv2.imread('../../data/many_cereals.jpg',0)
plt.imshow(many_box,cmap='gray')

## Oriented FAST and Rotated BRIEF (ORB)
- Brute Force Matching
- Distance metric

In [ ]:
# Intantiate orb object
orb = cv2.ORB_create()
# compute features
kp1,des1 = orb.detectAndCompute(one_box,None)
kp2,des2 = orb.detectAndCompute(many_box,None)
# match features : Brute Force
bf = cv2.BFMatcher(cv2.NORM_HAMMING,crossCheck=True)
matches = bf.match(des1,des2)
matches[0]



In [ ]:
# sorting key using lambda function that sorts based on distance member
matches = sorted(matches,key=lambda x:x.distance)
orb_matches = cv2.drawMatches(one_box,kp1,many_box,kp2,matches[:45],None,flags=2)
show_img(orb_matches)

# Scale invariant feature transform (SIFT)
- Brute Force
- KNN

In [ ]:
sift = cv2.SIFT_create()
# compute features
kp1,des1 = sift.detectAndCompute(one_box,None)
kp2,des2 = sift.detectAndCompute(many_box,None)
# match features : Brute Force
bf = cv2.BFMatcher()
matches = bf.knnMatch(des1,des2,k=2)
# len(des1)
len(matches)

In [ ]:
good = []
for match1,match2 in matches:
    if match1.distance < 0.6*match2.distance:
        good.append([match1])
# len(good)

In [ ]:
sift_matches = cv2.drawMatchesKnn(one_box,kp1,many_box,kp2,good,None,flags=2)
show_img(sift_matches)

# Scale invariant feature transform (SIFT)
- FLANN
- KNN

In [ ]:
sift = cv2.SIFT_create()
# compute features
kp1,des1 = sift.detectAndCompute(one_box,None)
kp2,des2 = sift.detectAndCompute(many_box,None)
# match features : FLANN
FLANN_INDEX_KDTREE = 0 
index_params = dict(algorithm = FLANN_INDEX_KDTREE,trees = 5)
search_params = dict(checks = 50)


In [ ]:
flann = cv2.FlannBasedMatcher(index_params,search_params)
matches = flann.knnMatch(des1,des2,k=2)


1. Create match mask, same length as matches, initialize to 0,0
2. Replace good array, by matches mask, flag pass value to 1,0 in mask
3. replace good array by matches, and pass in extra args as dict, along with color schema

In [ ]:
# Mask for plotting accurate color of match / no match

matchesMask = [[0,0] for i in range(len(matches))]
len(matchesMask)

In [ ]:
# good = []
for i,(match1,match2) in enumerate(matches):
    if match1.distance < 0.6*match2.distance:
        matchesMask[i]= [1,0]
        # good.append([match1])
len(matchesMask)

In [ ]:
draw_params = dict(matchColor = (0,255,0), 
                    singlePointColor=(255,0,0),
                    matchesMask = matchesMask, 
                    flags = 0)

In [ ]:
flann_matches = cv2.drawMatchesKnn(one_box,kp1,many_box,kp2,matches,None,**draw_params)
show_img(flann_matches)